In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import lightning as L
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from tqdm import tqdm  
import time 
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

This file uses LSTM model to predict the following week's feature(just danceability). The training process is using the 2022's data to train and the data of 2023 to validate(Same dataset as Holt-Winters), The validation MSE of danceability is 4.5530e-05, which is not as good as Holt-Winters(2e-05)...

In [2]:
df=pd.read_csv(r'C:\Users\Windows\Desktop\Spotify_Dataset_V3.csv',delimiter=';')
df_noURL=df.iloc[:,0:-1]
url=df.iloc[:,-1]
# Creat a daily average data of each feature
df_nodup = df_noURL.loc[:,['Title','Artists','Date','Danceability','Energy','Loudness','Speechiness','Acousticness','Instrumentalness','Valence']]
df_nodup = df_nodup.drop_duplicates(subset=['Title','Artists','Date'])
df_nodup = df_nodup.drop('Artists', axis=1)
df_nodup = df_nodup.iloc[::-1]
Date = [];Mean_dance=[];Mean_energy=[];Mean_loud=[];Mean_speech=[];Mean_acoustic=[];Mean_instru=[];Mean_valence=[]
Var_dance=[];Var_energy=[];Var_loud=[];Var_speech=[];Var_acoustic=[];Var_instru=[];Var_valence=[]
i=199;k=0
while (df_nodup.shape[0]-i)>0:
    date=datetime.strptime(df_nodup.iloc[i,1],'%d/%m/%Y').strftime('%Y-%m-%d')
    if i>200:
        if date == Date[-1]:
            k=k+1
            i=i+1
            continue
    Date.append(date)
    df_mean=df_nodup.iloc[i-199-k:i,2:].mean()
    df_var=df_nodup.iloc[i-199-k:i,2:].var()
    Mean_dance.append(df_mean.iloc[0]);Var_dance.append(df_var.iloc[0])
    Mean_energy.append(df_mean.iloc[1]);Var_energy.append(df_var.iloc[1])
    Mean_loud.append(df_mean.iloc[2]);Var_loud.append(df_var.iloc[2])
    Mean_speech.append(df_mean.iloc[3]);Var_speech.append(df_var.iloc[3])
    Mean_acoustic.append(df_mean.iloc[4]);Var_acoustic.append(df_var.iloc[4])
    Mean_instru.append(df_mean.iloc[5]);Var_instru.append(df_var.iloc[5])
    Mean_valence.append(df_mean.iloc[6]);Var_valence.append(df_var.iloc[6])
    i+=200
    k=0
'''
daily_mean = {'Date': Date, 'Mean_dance': Mean_dance,'Mean_energy':Mean_energy,'Mean_loud':Mean_loud,'Mean_speech':Mean_speech,'Mean_acoustic':Mean_acoustic,'Mean_instru':Mean_instru,'Mean_valence':Mean_valence}
dailymean_df = pd.DataFrame(data=daily_mean)
daily_var = {'Date': Date, 'Var_dance': Var_dance,'Var_energy':Var_energy,'Var_loud':Var_loud,'Var_speech':Var_speech,'Var_acoustic':Var_acoustic,'Var_instru':Var_instru,'Var_valence':Var_valence}
dailyvar_df = pd.DataFrame(data=daily_var)
Daily_mean = {'Mean_dance': Mean_dance,'Mean_energy':Mean_energy,'Mean_speech':Mean_speech,'Mean_acoustic':Mean_acoustic,'Mean_instru':Mean_instru,'Mean_valence':Mean_valence}
Dailydata = pd.DataFrame(data=Daily_mean)'''


"\ndaily_mean = {'Date': Date, 'Mean_dance': Mean_dance,'Mean_energy':Mean_energy,'Mean_loud':Mean_loud,'Mean_speech':Mean_speech,'Mean_acoustic':Mean_acoustic,'Mean_instru':Mean_instru,'Mean_valence':Mean_valence}\ndailymean_df = pd.DataFrame(data=daily_mean)\ndaily_var = {'Date': Date, 'Var_dance': Var_dance,'Var_energy':Var_energy,'Var_loud':Var_loud,'Var_speech':Var_speech,'Var_acoustic':Var_acoustic,'Var_instru':Var_instru,'Var_valence':Var_valence}\ndailyvar_df = pd.DataFrame(data=daily_var)\nDaily_mean = {'Mean_dance': Mean_dance,'Mean_energy':Mean_energy,'Mean_speech':Mean_speech,'Mean_acoustic':Mean_acoustic,'Mean_instru':Mean_instru,'Mean_valence':Mean_valence}\nDailydata = pd.DataFrame(data=Daily_mean)"

In [5]:
## Instead of coding an LSTM by hand, let's see what we can do with PyTorch's nn.LSTM()
class LightningLSTM(L.LightningModule):

    def __init__(self): # __init__() is the class constructor function, and we use it to initialize the Weights and Biases.
        
        super().__init__() # initialize an instance of the parent class, LightningModule.

        L.seed_everything(seed=41)
        
        ## input_size = number of features (or variables) in the data. In our example
        ##              we only have a single feature (value)
        ## hidden_size = this determines the dimension of the output
        ##               in other words, if we set hidden_size=1, then we have 1 output node
        ##               if we set hiddeen_size=50, then we hve 50 output nodes (that can then be 50 input
        ##               nodes to a subsequent fully connected neural network.
        self.lstm = nn.LSTM(input_size=1, hidden_size=1) 
         

    def forward(self, input):
        ## transpose the input vector
        for input_i in range(input.size(0)):
            input_trans = input[input_i].view(len(input[input_i]),1)
            lstm_out, temp = self.lstm(input_trans)
            ## lstm_out has the short-term memories for all inputs. We make our prediction with the last one
            if input_i==0:
                prediction=lstm_out[-1]
            else:
                prediction=torch.cat((prediction,lstm_out[-1]),0)
        #prediction = prediction
        return prediction
        
        
    def configure_optimizers(self): # this configures the optimizer we want to use for backpropagation.
        return Adam(self.parameters(), lr=0.001) ## we'll just go ahead and set the learning rate to 0.1

  
    def training_step(self, batch, batch_idx): # take a step during gradient descent.
        input_i, label_i = batch # collect input
        output_i = self.forward(input_i[0]) # run input through the neural network
        loss = ((output_i - label_i)**2).mean() ## loss = mean squared residual
        
        ###################
        ##
        ## Logging the loss and the predicted values so we can evaluate the training
        ##
        ###################
        self.log("train_loss", loss)
        '''
        if (label_i == 0):
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)
        '''
        return loss

In [6]:
#Dancibility
Train_data=[]
for i in range(31):
    train=Mean_dance[1823-31+i:2306-119-31+i]
    Train_data.append(train)#2022.1.1-2022.11.30-2022.12.30
Test_data=Mean_dance[2306-119-31:2306-119]#2022.12.1-2022.12.31
#Train_test_data=Mean_dance[1822:2306-119]#2022.1.1-2022.12.31
Validate_data_input=[]
for j in range(7):
    Validate_data_input.append(Mean_dance[2306-119+j:-7+j])#2023.1.1-2023.5.22
Validate_data_output=Mean_dance[-7:]                          
#Result_data=Mean_dance[2306-119:]#2023.1.1-2023.5.29 the data from model.result contains 7 days of pure prediction
float32_array = np.array(Train_data).astype(np.float32)
# Convert back to a list
Train_data = float32_array.tolist()
float32_array = np.array(Test_data).astype(np.float32)
# Convert back to a list
Test_data = float32_array.tolist()
float32_array = np.array(Validate_data_input).astype(np.float32)
# Convert back to a list
Validate_data_input = float32_array.tolist()
float32_array = np.array(Validate_data_output).astype(np.float32)
# Convert back to a list
Validate_data_output = float32_array.tolist()

In [7]:
model = LightningLSTM() # First, make model from the class

## print out the name and value for each parameter
print("Before optimization, the parameters are...")
for name, param in model.named_parameters():
    print(name, param.data)
    
print("\nNow let's compare the observed and predicted values...")
print(model(torch.tensor(Validate_data_input)))
print(Validate_data_output)

Seed set to 41


Before optimization, the parameters are...
lstm.weight_ih_l0 tensor([[-0.5271],
        [-0.5468],
        [ 0.6011],
        [-0.6616]])
lstm.weight_hh_l0 tensor([[-0.4701],
        [ 0.5440],
        [-0.7436],
        [ 0.4904]])
lstm.bias_ih_l0 tensor([0.6089, 0.2714, 0.1792, 0.3866])
lstm.bias_hh_l0 tensor([ 0.7565,  0.0814, -0.7200,  0.9227])

Now let's compare the observed and predicted values...
tensor([-0.0818, -0.0821, -0.0823, -0.0825, -0.0823, -0.0789, -0.0778],
       grad_fn=<CatBackward0>)
[0.6613568067550659, 0.6609396934509277, 0.6602814197540283, 0.660804033279419, 0.6709195971488953, 0.6735929846763611, 0.6614372134208679]


In [8]:
## create the training data for the neural network.
inputs = torch.tensor(Train_data)
labels = torch.tensor(Test_data)
dataset = TensorDataset(inputs.unsqueeze(0), labels.unsqueeze(0))
#dataset = TensorDataset(inputs, labels) 
dataloader = DataLoader(dataset)

In [9]:
## NOTE: Because we have set Adam's learning rate to 0.1, we will train much, much faster.
## Before, with the hand made LSTM and the default learning rate, 0.001, it took about 5000 epochs to fully train
## the model. Now, with the learning rate set to 0.1, we only need 300 epochs. Now, because we are doing so few epochs,
## we have to tell the trainer add stuff to the log files every 2 steps (or epoch, since we have to rows of training data)
## because the default, updating the log files every 50 steps, will result in a terrible looking graphs. So
trainer = L.Trainer(max_epochs=1000)#log_every_n_steps=2

trainer.fit(model, train_dataloaders=dataloader)

print("After optimization, the parameters are...")
for name, param in model.named_parameters():
    print(name, param.data)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
D:\Anaconda\envs\pytorch\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params | Mode 
--------------------------------------
0 | lstm | LSTM | 16     | train
--------------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0       

Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=1000` reached.


After optimization, the parameters are...
lstm.weight_ih_l0 tensor([[-0.3083],
        [-0.2816],
        [ 0.8590],
        [-0.4494]])
lstm.weight_hh_l0 tensor([[-0.0613],
        [ 0.9853],
        [-0.4530],
        [ 0.8947]])
lstm.bias_ih_l0 tensor([0.8275, 0.5370, 0.4374, 0.5966])
lstm.bias_hh_l0 tensor([ 0.9751,  0.3469, -0.4617,  1.1327])


In [10]:
print("\nlet's compare the observed and predicted values...")
print(model(torch.tensor(Validate_data_input)).detach())
print(Validate_data_output)


let's compare the observed and predicted values...
tensor([0.6613, 0.6605, 0.6598, 0.6592, 0.6588, 0.6606, 0.6629])
[0.6613568067550659, 0.6609396934509277, 0.6602814197540283, 0.660804033279419, 0.6709195971488953, 0.6735929846763611, 0.6614372134208679]


In [11]:
mse=((model(torch.tensor(Validate_data_input)).detach()-torch.tensor(Validate_data_output))**2).mean()
print('Validation MSE for a week')
print(mse)

Validation MSE for a week
tensor(4.5530e-05)


In [4]:
def LSTM_weekpredict(data, model):
    """
        Returns prediction in the following week
    """
    # errors array
    predict=[]
    for i in range(7):
        pred = append(model(torch.tensor(data)).detach())
        data = np.append(data,pred)
        predict= predict.append(pred)
 
    return np.array(predict)  # mean errors